In [1]:
from hwt.synthesizer.interface import Interface
from hwt.interfaces.std import VectSignal, Signal
from hwt.synthesizer.unit import Unit
from hwtLib.amba.axis import AxiStreamAgent
from hwt.hdl.constants import DIRECTION

# An Interface is a base class of HWT interfaces, it's usage is straight forward
# Note that hwtLib.amba.axis already contains definition of AxiStream
class AxiStream(Interface):
    """
    An example of interface definition in HWT
    (AMBA4 AXI-4 Stream https://static.docs.arm.com/ihi0051/a/IHI0051A_amba4_axi4_stream_v1_0_protocol_spec.pdf)
    """
    def _declr(self):
        """
        Interface has the _declr() method with same meaning as Unit._declr shown in previous tutorial
        it is the place where publically visible interfaces should be declared.
        """
        DATA_WIDTH = 64
        # self.<interface name> = <interface object>
        self.data = VectSignal(DATA_WIDTH)
        self.strb = VectSignal(DATA_WIDTH//8)
        self.last = Signal()
        self.valid = Signal()
        self.ready = Signal(masterDir=DIRECTION.IN) # ready will be input to master

    def _getIpCoreIntfClass(self):
        """
        An optional method where you can override how the interface should be represented in exported IP-cores
        """
        return IP_AXIStream

    def _initSimAgent(self, sim):
        """
        An optional method where you can override the simulation agent used is simulation
        to read/write from/to interface
        """
        # the ._ag has to be specified otherwise the simulator won't be able to communicate
        # with the circuit if this inteface is on top component
        self._ag = AxiStreamAgent(sim, self)


from hwt.interfaces.utils import addClkRstn
# simple wire with our interface
class AxiStreamWire(Unit):

    def _declr(self):
        # addClkRstn just adds self.clk = Clk(); self.rst_n = Rst_n()
        # we are adding it because the AxiStreamAgent needs it as AxiStream is synchronous interface
        addClkRstn(self) 
        self.a = AxiStream()
        self.b = AxiStream()._m()

    def _impl(self):
        self.b(self.a)


class AxiStreamWireSignalBySignal(AxiStreamWire):

    def _impl(self):
        a, b = self.a, self.b
        b.data(a.data)
        b.strb(a.strb)
        b.last(a.last)
        b.valid(a.valid)
        # note that there the direction is reversed
        # because the direction in AxiStream definition is reversed as well
        a.ready(b.ready)


class AxiStreamWireSignalBySignal2(AxiStreamWire):

    def _impl(self):
        # each Interface/Unit instance has _interface list of children which can be
        # used for introspection etc.
        for a, b in zip(self.a._interface, self.b._interface):
            if a is self.a.ready:
                a(b)
            else:
                b(a)

In [2]:
from hwt.hdl.constants import Time
from hwt.simulator.simTestCase import SimTestCase
from pycocotb.constants import CLK_PERIOD
from pyMathBitPrecise.bit_utils import mask

# An example simulation with our interface
class AxiStreamWireWireTC(SimTestCase):

    def test_simple(self):
        u = AxiStreamWire()
        self.compileSimAndStart(u)
        # The data format depends on implementation of simulation agent
        # and it can be found in documentation of the agent.
        # It is usually a tuple of integers corresponding to values of signals
        # in order in which they are defined.
        inputData = [
            # (data, strb, last) as AxiStreamAgent requires
            (i, mask(64//8), 1)
            for i in range(5)
        ]
        u.a._ag.data.extend(inputData)
        self.runSim(10 * CLK_PERIOD)
        self.assertValSequenceEqual(u.b._ag.data, inputData)


In [3]:
from jupyter_widget_hwt import HwtSignalDumpWidget

selected_test = AxiStreamWireWireTC('test_simple')
trace = HwtSignalDumpWidget(selected_test, width=1000, height=500)
display(trace)

HwtSignalDumpWidget(height='500px', value={'name': 'AxiStreamWireWireTC_test_simple__AxiStreamWire', 'type': {…